In [1]:
import numpy as np
import matplotlib.pyplot as plt
from transformers import BertTokenizer
from torch.utils.data import DataLoader
import pandas as pd


#### Ejemplos con BertTokenizer

In [2]:
url = 'https://github.com/LeoBrasileo/Aprendizaje-Automatico-TP2/raw/refs/heads/main/data/datos_google_argento.csv'
df_google_arg = pd.read_csv(url, sep=',', index_col=False)
df_google_arg.head()

,texto_original,texto_limpio
0,"Para la caída del cabello, tengo un nuevo champú",para la caída del cabello tengo un nuevo champú
1,Los hámsters comen zanahorias,los hámsters comen zanahorias
2,¿Me podés mandar fotos de la pileta?,me podés mandar fotos de la pileta
3,Hola Cristina que bueno que pueda hablar con vos,hola cristina que bueno que pueda hablar con vos
4,Tranquilo va a estar todo bien,tranquilo va a estar todo bien


Declaramos a nuestro tokenizer como el tokenizer de Bert

In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# Defino el tamaño del vocabulario como el tamaño del vocabulario default de bert-base-multilingual-cased,
# Esto se usaría en las capas de embeddings, como dimensión. 
VOCAB_SIZE = tokenizer.vocab_size

In [4]:
original = '¿A qué hora vamos a McDonalds?'
texto = "a qué hora vamos a mcdonalds"
tokens = tokenizer.tokenize(texto)
tokens_orig = tokenizer.tokenize(original)

print(tokens)
print(tokens_orig)

['a', 'qué', 'hora', 'va', '##mos', 'a', 'm', '##c', '##dona', '##ld', '##s']
['¿', 'A', 'qué', 'hora', 'va', '##mos', 'a', 'McDonald', '##s', '?']


In [5]:
print(tokenizer.convert_tokens_to_ids(tokens))

[169, 38188, 24301, 10321, 13386, 169, 181, 10350, 64674, 12620, 10107]


### Limpieza de strings

In [6]:
vocales_con_acento_min = [chr(x) for x in [ord('á'), ord('é'), ord('í'), ord('ó'), ord('ú')]]
vocales_con_acento_may = [chr(x) for x in [ord('Á'), ord('É'), ord('Í'), ord('Ó'), ord('Ú')]]
allowed_characters = [chr(x) for x in range(97,123)] + vocales_con_acento_min + [chr(241)] + [chr(10), chr(32)] + [chr(x) for x in range(48,58)] # los caracteres de letras minúsculas, espacio en blanco y fin de linea y números
characters_to_replace = [chr(x) for x in range(65,91)] + vocales_con_acento_may + [chr(209)] #las mayúsculas
characters_to_replace_with = [chr(x) for x in range(97,123)] + vocales_con_acento_min + [chr(241)] #las minúsculas
replace_dict = dict(zip(characters_to_replace, characters_to_replace_with))

def limpiar_string (s) :
  res = ""
  for c in s :
    if c in allowed_characters :
      res += c
    elif c in characters_to_replace :
      res += replace_dict[c]
  return res

In [7]:
test_limpieza = "Veamos que pasa si cortamos un árbol ¿Será capaz de entender? El Árbol no sentirá el corte hasta que sea demaisado tarde: al buscar sus raices para nutrir las hojas, sentirá la desesperación de la falta." + '\n' + '¡Ósado el talador, que se preocupa más por el calor de su morada que la protección que ofrece el bosque y la protección de todas sus criaturas contra el silencio verdadero!'
print(test_limpieza)
print('\n')
print(limpiar_string(test_limpieza))

Veamos que pasa si cortamos un árbol ¿Será capaz de entender? El Árbol no sentirá el corte hasta que sea demaisado tarde: al buscar sus raices para nutrir las hojas, sentirá la desesperación de la falta.
¡Ósado el talador, que se preocupa más por el calor de su morada que la protección que ofrece el bosque y la protección de todas sus criaturas contra el silencio verdadero!


veamos que pasa si cortamos un árbol será capaz de entender el árbol no sentirá el corte hasta que sea demaisado tarde al buscar sus raices para nutrir las hojas sentirá la desesperación de la falta
ósado el talador que se preocupa más por el calor de su morada que la protección que ofrece el bosque y la protección de todas sus criaturas contra el silencio verdadero


### Etiquetas para datos de entrada

**Nota**: es un planteo preliminar, imitando cómo nos piden entregar las respuestas, se pueden hacer modificaciones para lo que necesitemos como entrada

In [8]:
import pandas as pd
from py.funciones_dataset import asignar_etiquetas_puntuacion, asignar_puntuacion_a_tokens, reconstruir_texto

##### Ejemplo 1:

Primero definimos puntuación para los tokens

Este ejemplo, usando el - , no anda

In [9]:
texto_original = "¿Cuándo vamos a McDonald's?Ellos no vienen. Entonces, - ¿dónde están?"

texto_limpio = limpiar_string(texto_original)
tokens_texto_limpio = tokenizer.tokenize(texto_limpio)

puntuacion_tokens = asignar_puntuacion_a_tokens(instancia_original=texto_original,
                                                instancia_id=1,
                                                instancia_tokens=tokens_texto_limpio)
pd.DataFrame.from_dict(puntuacion_tokens)

,instancia_id,token,token_id,capitalizacion,puntuacion_inicial,puntuacion_final
0,1,cu,10854,1,¿,
1,1,##ánd,101439,1,,
2,1,##o,10133,1,,
3,1,va,10321,0,,
4,1,##mos,13386,0,,
5,1,a,169,0,,
6,1,m,181,2,,
7,1,##c,10350,2,,
8,1,##dona,64674,2,,
9,1,##ld,12620,2,,


La instancia etiquetada final (con las etiquetas definidas en _ETIQUETA_PUNT_FINAL_ y _ETIQUETA_PUNT_INICIAL_) sería de esta forma:

In [10]:
tokens_etiquetados = asignar_etiquetas_puntuacion(puntuacion_tokens)
pd.DataFrame.from_dict(tokens_etiquetados)

,instancia_id,token,token_id,capitalizacion,puntuacion_inicial,puntuacion_final
0,1,cu,10854,1,1,0
1,1,##ánd,101439,1,0,0
2,1,##o,10133,1,0,0
3,1,va,10321,0,0,0
4,1,##mos,13386,0,0,0
5,1,a,169,0,0,0
6,1,m,181,2,0,0
7,1,##c,10350,2,0,0
8,1,##dona,64674,2,0,0
9,1,##ld,12620,2,0,0


##### Ejemplo 2:

Este ejemplo anda, eliminando el -. 

In [11]:
texto_original = "¿Cuándo vamos a McDonald's? Ellos no vienen. Entonces, ¿dónde están?"

texto_limpio = limpiar_string(texto_original)
tokens_texto_limpio = tokenizer.tokenize(texto_limpio)

puntuacion_tokens = asignar_puntuacion_a_tokens(instancia_original=texto_original,
                                                instancia_id=1,
                                                instancia_tokens=tokens_texto_limpio)

tokens_etiquetados = asignar_etiquetas_puntuacion(puntuacion_tokens)

pd.DataFrame.from_dict(tokens_etiquetados)

,instancia_id,token,token_id,capitalizacion,puntuacion_inicial,puntuacion_final
0,1,cu,10854,1,1,0
1,1,##ánd,101439,1,0,0
2,1,##o,10133,1,0,0
3,1,va,10321,0,0,0
4,1,##mos,13386,0,0,0
5,1,a,169,0,0,0
6,1,m,181,2,0,0
7,1,##c,10350,2,0,0
8,1,##dona,64674,2,0,0
9,1,##ld,12620,2,0,0


##### Ejemplo 3:

In [12]:
tokens_test_limpieza = tokenizer.tokenize(limpiar_string(test_limpieza))

puntuacion_tokens = asignar_puntuacion_a_tokens(instancia_original=test_limpieza,
                                                instancia_id=1,
                                                instancia_tokens=tokens_test_limpieza)

tokens_etiquetados = asignar_etiquetas_puntuacion(puntuacion_tokens)

pd.DataFrame.from_dict(tokens_etiquetados).head(12)

,instancia_id,token,token_id,capitalizacion,puntuacion_inicial,puntuacion_final
0,1,ve,10323,1,0,0
1,1,##amos,73983,1,0,0
2,1,que,10121,0,0,0
3,1,pasa,26088,0,0,0
4,1,si,10294,0,0,0
5,1,corta,53134,0,0,0
6,1,##mos,13386,0,0,0
7,1,un,10119,0,0,0
8,1,árbol,55220,0,0,0
9,1,será,23843,1,1,0


# Clase del dataset de 100 Años de Soledad


In [13]:
#creaar DataFrame para alimentar el dataset (pensando en que va a ser mas rapido para el entrenamiento)
data_path ='https://github.com/LeoBrasileo/Aprendizaje-Automatico-TP2/raw/refs/heads/main/data/marquez_cleaned.csv'  # Remove the leading slash
data = pd.read_csv(data_path, delimiter=',')
texto_original = data['texto_original'].tolist()
# texto_limpio = data['texto_limpio'].tolist()

puntuacion_tokens_marquez = []
problematic_sentences = []

for i in range(4,len(texto_original)):
    try:
        # tokens_texto_original = tokenizer.tokenize(texto_original[i])
        texto_limpio = limpiar_string(texto_original[i])
        tokens_texto_limpio = tokenizer.tokenize(texto_limpio)

        puntuacion_tokens = asignar_puntuacion_a_tokens(instancia_original=texto_original[i],
                                                        instancia_id=i+1,
                                                        instancia_tokens=tokens_texto_limpio)
        puntuacion_tokens = asignar_etiquetas_puntuacion(puntuacion_tokens)
        puntuacion_tokens_marquez.append(puntuacion_tokens)
    except Exception as e:
        # Store problematic sentence info instead of breaking
        problematic_sentences.append({
            'index': i,
            'original_text': texto_original[i],
            'cleaned_text': texto_limpio if 'texto_limpio' in locals() else None,
            'error': str(e)
        })
        continue


print(f"funcionaron: {len(puntuacion_tokens_marquez)} sentences")
print(f"con problemas: {len(problematic_sentences)}")

# oraciones con problemas
for prob in problematic_sentences[:5]:  # Show first 5 problematic sentences
    print(f"Index {prob['index']}: {repr(prob['original_text'][:100])}... Error: {prob['error']}")


# renumerar para que los ids de instancia sean consecutivos
puntuacion_tokens_marquez_clean = []
for new_id, tokens_list in enumerate(puntuacion_tokens_marquez):
    tokens_list_updated = []
    for token_dict in tokens_list:
        token_dict_updated = token_dict.copy()
        token_dict_updated['instancia_id'] = new_id + 1  # que empiece en 1?
        tokens_list_updated.append(token_dict_updated)
    puntuacion_tokens_marquez_clean.append(tokens_list_updated)


funcionaron: 1810 sentences
con problemas: 336
Index 19: 'Si has de volverte loco, vuélvete tú solo gritó . Pero no trates de inculcar a los niños tus ideas d'... Error: string index out of range
Index 28: 'Es el olor del demonio dijo ella. En absoluto corrigió Valeria . Está comprobado que el demonio tien'... Error: string index out of range
Index 36: 'En el mundo están ocurriendo cosas increíbles le decía a Gabriel . Ahí mismo, al otro lado del río, '... Error: string index out of range
Index 51: 'No importa decía Vega . Lo esencial es no perder la orientación.'... Error: string index out of range
Index 58: 'Sus sueños terminaban frente a ese mar color de ceniza, espumoso y sucio, que no merecía los riesgos'... Error: string index out of range


In [14]:
from py.marquez_clean import MarquezDataset, marquez_collate_fn

dataset_marquez = MarquezDataset(data=puntuacion_tokens_marquez)
loader = DataLoader(dataset_marquez, batch_size=32, shuffle=False, collate_fn=marquez_collate_fn)

##### Ejemplo de juguete con 10 samples

Entrenamos con muchas epochs y pocos samples, para ver si la red logra memorizarlos. Principalmente lo dejo para que lo usemos para verificar que las funciones se ejecutan bien (en este caso en particular, usando la RNN clásica, pero se puede usar en otro momento para cuando haya cambios en las redes)

In [30]:
from py.funciones_entrenamiento import entrenar_modelo
from py.modelos.RNN_Clasica import RNN_Clasica
import torch
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'xpu:0' if torch.xpu.is_available() else device # placa intel

In [29]:
NUM_EPOCHS_UNI = 150
LEARNING_RATE_UNI = 0.0001
EMBEDDING_DIM_UNI = 64
HIDDEN_UNI = 128

modelo_unidireccional = RNN_Clasica(hidden_size=HIDDEN_UNI, vocab_size=VOCAB_SIZE, embedding_dim=EMBEDDING_DIM_UNI)
modelo_unidireccional = modelo_unidireccional.to(device)

optimizer_unidireccional = torch.optim.Adam(modelo_unidireccional.parameters(), lr=LEARNING_RATE_UNI)
criterion_unidireccional = nn.CrossEntropyLoss()

print(f"Modelo de red neuronal unidireccional creado con {sum(p.numel() for p in modelo_unidireccional.parameters())} parámetros")

Modelo de red neuronal unidireccional creado con 7883722 parámetros


In [16]:
# Tomamos 10 samples del dataset de Gabriel García Marquez
dataset_marquez_reduced = MarquezDataset(data=puntuacion_tokens_marquez[:10])

# DataLoader con batches de tamaño 1 (dado que son solo 10 samples, pero también se podría hacer 1 solo batch de 10 secuencias, 2 batches de 5 secuencias, o tomar más samples)
loader = DataLoader(dataset_marquez_reduced, batch_size=1, shuffle=True, collate_fn=marquez_collate_fn)

In [31]:
loader = DataLoader(dataset_marquez_reduced, batch_size=1, shuffle=True, collate_fn=marquez_collate_fn)

_ , _ = entrenar_modelo(modelo_unidireccional, loader, loader, optimizer_unidireccional, criterion_unidireccional, cant_epochs=200, device=device)

Iniciando entrenamiento...
--------------------------------------------------


KeyboardInterrupt: 